In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt1_transe', monitor='val_mrr', verbose=0, 
                                                save_best_only=True, mode='min')

dataset = load_fb15k_237()

model.fit(dataset['train'],
          batch_size=10000,
          epochs=10,
          validation_freq=2,
          validation_batch_size=100,
          validation_burn_in=2,
          validation_data = dataset['valid'][::100],
          callbacks=[checkpoint])     # Pass the callback to the fit function


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Epoch 1/10
29/29 [==============================] - 3s 101ms/step - loss: 6625.2280
Epoch 2/10
29/29 [==============================] - 3s 92ms/step - loss: 6505.0376 - val_mrr: 0.0576 - val_mr: 3106.6989 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1562 - val_hits@100: 0.3352
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 3/10
29/29 [==============================] - 1s 49ms/step - loss: 6386.7886
Epoch 4/10
29/29 [==============================] - 2s 63ms/step - loss: 6267.6465 - val_mrr: 0.0673 - val_mr: 2225.0398 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1903 - val_hits@100: 0.3835
Epoch 5/10
29/29 [==============================] - 1s 47ms/step - loss: 6149.4019
Epoch 6/10
29/29 [==============================] - 2s 63ms/step - loss: 6029.9697 - val_mrr: 0.0765 - val_mr: 1584.9233 - val_hits@1: 0.000

In [4]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 407s 2s/step
MR: 921.3923084450533
MRR: 0.17730360323410402
hits@1: 0.12151384675604267
hits@10: 0.2851551032390645


In [5]:
from ampligraph.utils import save_model
# explictly save the model
save_model(model, 'saved_model_transE')


WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [6]:
from ampligraph.utils import restore_model

# restore saved models or checkpoints
model = restore_model('saved_model_transE')

Saved model does not include a db file. Skipping.


In [7]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 254s 1s/step
MR: 921.3923084450533
MRR: 0.17730360323410402
hits@1: 0.12151384675604267
hits@10: 0.2851551032390645
